In [1]:
import sys
sys.path.append('/home/dboiko/simg-2')

import os
os.chdir('/home/dboiko/simg-2/experiments/downstream_model/additional_benchmarks/qm7')

In [2]:
from model import GNN
import pandas as pd
import torch
from torch_geometric.data import Data, Batch
from tqdm.auto import tqdm
import numpy as np

/home/dboiko/.conda/envs/nbo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [197]:
CHECKPOINT_SUFFIX = "/lightning_logs/version_0/checkpoints/"

results = []

for target in ["atomization","dipole","gap","homo"]:
    for fraction in [0.2, 0.4, 0.6, 0.8, 1.0]:
        for attempt in range(3):
            log_path = f'logs_long/{target}_{fraction:.1f}_{attempt}'

            checkpoints = os.listdir(log_path+CHECKPOINT_SUFFIX)
            checkpoints = list(filter(lambda x: x.startswith('best_val'), checkpoints))

            val_losses = list(map(lambda x: float(x.split('-')[1].split('=')[1]), checkpoints))
            best_checkpoint = checkpoints[np.argmin(val_losses)]

            model = GNN.load_from_checkpoint(log_path + CHECKPOINT_SUFFIX + best_checkpoint).eval().cuda()

            file2y = np.loadtxt(f'/home/dboiko/SPAHM/SPAHM/1_QM7/target/{target}.dat')
            test_idx = set(list(np.loadtxt('/home/dboiko/SPAHM/test-indices/QM7_test_indices.dat').astype(int)))

            data = torch.load('/home/dboiko/reproducing_experiments/qm7_graphs_from_repo.pt')
            nice_graphs = []

            for graph in tqdm(data):
                nice_graphs.append(
                    Data(
                        x=torch.FloatTensor(graph.x),
                        y=torch.FloatTensor([file2y[int(graph.qm9_id.split('_')[1])]]),
                        edge_index=torch.LongTensor(graph.edge_index),
                        edge_attr=torch.FloatTensor(graph.edge_attr),
                        type='train' if int(graph.qm9_id.split('_')[1]) not in test_idx else 'test'
                    )
                )

            test = [graph for graph in nice_graphs if graph.type == 'test']
            huge_batch = Batch.from_data_list(test).cuda()

            preds = model(huge_batch.x, huge_batch.edge_index, huge_batch.edge_attr, huge_batch.batch).cpu().detach().numpy()

            y_mean, y_std = np.load(log_path+'/mean_std.npy')
            mae = np.abs(preds[:, 0]*y_std + y_mean - huge_batch.y.cpu().detach().numpy()).mean()

            results.append({
                'target': target,
                'fraction': fraction,
                'attempt': attempt,
                'mae': mae
            })

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7165/7165 [00:01<00:00, 5884.72it/s]


In [188]:
(len(nice_graphs)-len(test))*0.8

4585.6

In [198]:
results = pd.DataFrame(results)

In [200]:
results.to_csv('results_SPAHM_comparison_longer.csv')

In [184]:
!pwd

/home/dboiko/simg-2/experiments/downstream_model/additional_benchmarks/qm7
